# Erstellen Energiebedarszeitreihe der Ursprungsdaten zum Validieren der Simulation

In [1]:
import pandas as pd
import numpy as np 
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Globalparameter

In [2]:
# Tagtyp: 1 = Werktag, 2 = Samstag, 3 = Sonntag
TYPE_D = 1 
# Breite der Zeitschritte
T_STEP = 15 
# Ladeszenario: 1 = nur Zuhause, 2 = Zuhause und Arbeit, 3 = Überall
CHARGE_SCEN = 3
# Ladeleistung 
PCHARGE_SLOW = 3.7 
# Abspeichern der simulierten Fahrzeuge
simulated_evs = []

In [3]:
class Electric_Vehicle(object):
    model_df = pd.read_excel(r"C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\EV_Modelle_Tabelle.xlsx", index_col='Modell')
    SEGMENTS = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -9])
    # Dummy Wahrscheinlichkeiten aus MOP Studie
    PROB_SEGMENT = [0.0592, #1
        0.1958, #2
        0.2621, #3
        0.1597, #4
        0.029, #5
        0.0058, #6
        0.0354, #7
        0.0122, #8
        0.0637, #9
        0.0586, #10
        0.0386, #11
        0.0058, #12
        0.0528, #13
        0.0213 #-9
        ]

    
    def __init__(self):
        segment = self.choose_segment()
        model = self.choose_model(segment)
        self.MODEL = model
        
        # Bug da Modelle mehrfach in Liste keine eindeutige Zuordnung sondern Array als output -> fixed -> elegantere Lösung?
        # Segment des Fahrzeugs
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Segment"]):
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"]
        else:
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"][0]
        
        # Batteriekapazität in kWh
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Batterie"]):
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"]
        else:
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"][0]
        
        # Verbrauch in kWh/100km
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Verbrauch"]):
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"]
        else:
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"][0]
        
        # Leistung -> aktuell nicht gebraucht
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Leistung"]):
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"]
        else:
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"][0]
            
        self.SOC = 100
        self.trip_no = 0
        self.trips = []
    
    def choose_segment(self): 
        # wähle p zufällig auf Basis gegebener W'keiten
        choice = np.random.choice(Electric_Vehicle.SEGMENTS, p=Electric_Vehicle.PROB_SEGMENT)
        
        # Vorerst Wahl des häufigsten Semgents bei keiner Angabe -> später überarbeiten
        if choice == -9: 
            choice = 3
        
        return choice
    
    def choose_model(self, segment):
        models = Electric_Vehicle.model_df
        
        # filtern der infragekommenden Fahrzeuge über Segment
        filt = models["Segment"] == segment
        choices = models[filt]
        
        # Wahl einse zufälligen Fahrzeugs aus der Liste
        pick = np.random.randint(0, len(choices))
        model = choices.index[pick]
        return model
    
    # drive() und charge() testen
    
    def drive(self, distance):
        trip_consumption = distance * (self.CONSUMPTION / 100) 
        self.SOC = self.SOC - (trip_consumption / self.CAPACITY) * 100
                
    
    def check_distance(self, distance):
        # überprüfe SOC für gegebene Distanz ausreichend ist
        trip_consumption = distance * (self.CONSUMPTION / 100)
        if trip_consumption < self.SOC/100 * self.CAPACITY:
            return True 
        else:
            return False
        
    def max_distance(self):
        # maximale Distanz die mit aktuellem SOC zurückgelegt werden kann
        max_dist = round(((self.SOC/100)*self.CAPACITY / self.CONSUMPTION) * 100, 2)
        return max_dist
        
    def charge(self):       
        # Fahrzeug befindet sich im Zielzustand des letzten Trips
        trip = self.trips[len(self.trips)-1]
        # nötige Zeit zum vollständigen Aufladen in Minuten
        t_charge_full = (((100 - ev.SOC)/100 * ev.CAPACITY) / PCHARGE_SLOW) * 60
        # Ladezeit beschränkt durch Zeit zum vollständigen Aufladen und Länge des Aufenthalts
        t_charge = min(t_charge_full, trip.stay_duration)
        # update SOC über Ladedauer und Ladeleistung
        self.SOC = self.SOC + ((t_charge/60 * PCHARGE_SLOW) / self.CAPACITY) * 100
        trip.charge_start = int(trip.arrival)
        trip.charge_end = int(round(trip.charge_start + t_charge))

In [4]:
class Trip(object):
    
    def __init__(self, whyfrom, departure):
        self.trip_id = None
        self.trip_no = None
        self.whyfrom = whyfrom
        self.whyto = None
        self.departure = departure
        self.departure_t = round(float(departure/T_STEP))
        self.arrival = None
        self.trip_duration = None
        self.distance = None
        self.stay_duration = None
        self.SOC_start = None
        self.SOC_end = None
        self.charge_start = None 
        self.charge_end = None

In [43]:
# einlesen des bearbeiteten Trips-Datensatz (mit errechneten Aufenhtalten)
# bereits auf Wochentage gefiltert
df = pd.read_csv(r'C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\NHTS_trips_processed_WT_Aufenthalt.csv')

In [44]:
df.head()

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
0,300000071,1,3,1,1,4,8.439,15,1000,1015,600,615,40,295
1,300000071,1,3,2,4,1,8.286,20,1510,1530,910,930,61,1110
2,300000072,1,6,1,2,1,135.191,120,700,900,420,540,28,540
3,300000072,1,6,2,1,2,131.367,150,1800,2030,1080,1230,72,630
4,300000073,1,3,1,1,2,3.621,15,845,900,525,540,35,330


In [45]:
df = df.sort_values(by=["ID", "Trip_no"])

In [46]:
df.tail(20)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
633001,407943012,1,6,5,3,1,7.689,12,1722,1734,1042,1054,69,866
633002,407943013,1,5,1,1,5,14.669,26,1631,1657,991,1017,66,1
633003,407943013,1,5,2,5,3,1.257,7,1658,1705,1018,1025,68,53
633004,407943013,1,5,3,3,5,1.257,8,1758,1806,1078,1086,72,3
633005,407943013,1,5,4,5,1,15.389,29,1809,1838,1089,1118,73,1313
633006,407943016,1,5,1,1,4,-1.000,26,1631,1657,991,1017,66,72
633007,407943016,1,5,2,4,1,-1.000,29,1809,1838,1089,1118,73,1313
633008,3002373510,5,5,1,4,1,-1.000,690,700,1830,420,1110,28,750
633009,3002373511,5,5,1,4,1,-1.000,690,700,1830,420,1110,28,750
633010,3046754510,2,5,1,1,5,-1.000,10,1400,1410,840,850,56,5


In [47]:
df.shape

(633021, 14)

## Analoge Simulation der EVs / SOCs und Ladevorgänge zur Fahrtensimulation auf Basis originaler Trips

In [49]:
total_evs = []
#rows = df_test.shape[0] 
ev = None
for row in range(df.shape[0]):
    if (df.at[row, "Trip_no"]) == 1 or (df.at[row, "ID"] != df.at[row-1, "ID"]):
      
        if ev:
            if ev.SOC < 100:
                location = ev.trips[len(ev.trips)-1].whyto
                # obere Schranke der Ladezeit, tatsächliche wird in ev.charge() bestimmt
                duration = ev.trips[len(ev.trips)-1].stay_duration
                if CHARGE_SCEN == 1:
                    # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
                    if location == 1 and duration > 15:
                        ev.charge()

                elif CHARGE_SCEN == 2:
                    # geladen wird nur wenn sich das Fahrzeug Zuhause oder auf der Arbeit befindet und sich dort länger als 15 Minuten aufhält
                    if (location == 1 or location == 2) and duration > 15:
                        ev.charge()

                elif CHARGE_SCEN == 3:
                    # geladen wird in jedem Zustand, sofern die Parkdauer 15 Minuten übersteigt
                    if duration > 15:
                        ev.charge()   
            total_evs.append(ev)
            
        ev = Electric_Vehicle()
    trip = Trip(df.at[row, "Whyfrom"], df.at[row, "Departure"])
    trip.trip_id = df.at[row, "ID"]
    trip.trip_no = df.at[row, "Trip_no"]
    trip.whyto = df.at[row, "Whyto"]
    trip.arrival = df.at[row, "Arrival"]
    trip.trip_duration = df.at[row, "Trip_duration"]
    trip.distance = df.at[row, "Distance"]
    trip.stay_duration = df.at[row, "Stay_duration"]
    
    if ev.SOC < 100:
        location = ev.trips[len(ev.trips)-1].whyto
        # obere Schranke der Ladezeit, tatsächliche wird in ev.charge() bestimmt
        duration = ev.trips[len(ev.trips)-1].stay_duration
        if CHARGE_SCEN == 1:
            # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
            if location == 1 and duration > 15:
                ev.charge()

        elif CHARGE_SCEN == 2:
            # geladen wird nur wenn sich das Fahrzeug Zuhause oder auf der Arbeit befindet und sich dort länger als 15 Minuten aufhält
            if (location == 1 or location == 2) and duration > 15:
                ev.charge()

        elif CHARGE_SCEN == 3:
            # geladen wird in jedem Zustand, sofern die Parkdauer 15 Minuten übersteigt
            if duration > 15:
                ev.charge()    
            
    trip.SOC_start = round(ev.SOC, 1)
    # Fahrvorgang -> update SOC 
    ev.drive(trip.distance)
    trip.SOC_end = round(ev.SOC, 1)
    ev.trips.append(trip)
    


In [51]:
from collections import defaultdict

# erzeugen Dictionary "Key : List"
total_trips_dict = defaultdict(list)

# speichern der Trips jedes einzelnen Fahrzeugs im dict
for ev in total_evs:
    for trip in ev.trips:
        # .__dict__.items() returns Dictionary mit allen Member Variablen und dazugehörigen Werten des Objekts
        for key, val in trip.__dict__.items():
            total_trips_dict[key].append(val)

# umwandeln in DataFrame
final_data = pd.DataFrame(total_trips_dict)

In [26]:
data.head(20)

,trip_id,trip_no,whyfrom,whyto,departure,departure_t,arrival,trip_duration,distance,stay_duration,SOC_start,SOC_end,charge_start,charge_end
0,300000071,1,1,4,600,40,615,15,8.439,295,100.0,96.5,615.0,631.0
1,300000071,2,4,1,910,61,930,20,8.286,1110,100.0,96.6,930.0,945.0
2,300000072,1,2,1,420,28,540,120,135.191,540,100.0,73.0,540.0,978.0
3,300000072,2,1,2,1080,72,1230,150,131.367,630,100.0,73.7,1230.0,1656.0
4,300000073,1,1,2,525,35,540,15,3.621,330,100.0,99.0,540.0,547.0
5,300000073,2,2,1,870,58,885,15,3.610,1080,100.0,99.0,885.0,892.0
6,300000081,1,1,2,675,45,690,15,12.902,720,100.0,96.5,690.0,713.0
7,300000081,2,2,1,1410,94,1420,10,12.902,695,100.0,96.5,1420.0,1443.0
8,300000121,1,1,4,350,23,365,15,5.464,55,100.0,98.5,365.0,383.0
9,300000121,2,4,1,420,28,435,15,5.464,15,100.0,98.5,NaN,NaN


In [52]:
final_data.shape

(633015, 14)

In [53]:
df.shape

(633021, 14)

In [55]:
# Spaltennamen umformatieren
final_data = final_data.rename(str.capitalize, axis='columns')
final_data.head()

,Trip_id,Trip_no,Whyfrom,Whyto,Departure,Departure_t,Arrival,Trip_duration,Distance,Stay_duration,Soc_start,Soc_end,Charge_start,Charge_end
0,300000071,1,1,4,600,40,615,15,8.439,295,100.0,93.7,615.0,633.0
1,300000071,2,4,1,910,61,930,20,8.286,1110,100.0,93.8,930.0,948.0
2,300000072,1,2,1,420,28,540,120,135.191,540,100.0,61.9,540.0,849.0
3,300000072,2,1,2,1080,72,1230,150,131.367,630,100.0,63.0,1230.0,1530.0
4,300000073,1,1,2,525,35,540,15,3.621,330,100.0,98.5,540.0,547.0


## Warum 6 Trips weniger?

In [56]:
import pickle

path = r"C:\Users\thoma\Desktop\ev-modelling-repo\Datenauswertung\Werktag\Simulationsdaten\NHTS_Trips_Werktag_CS3_Auswertung.pickle"

pickle.dump(final_data, open(path,"wb"))

In [58]:
df.sort_values(by=["Trip_no"], ascending=False)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
288458,304679941,5,3,50,4,1,11.375,19,2355,14,1435,14,96,1126
288457,304679941,5,3,49,1,4,12.250,15,2021,2036,1221,1236,81,199
280500,304555151,2,6,47,2,1,4.514,13,1825,1838,1105,1118,74,742
288456,304679941,5,3,47,2,2,5.852,12,1922,1934,1162,1174,77,47
288455,304679941,5,3,46,2,2,13.559,11,1910,1921,1150,1161,77,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346247,400092742,1,3,1,1,2,10.361,20,710,730,430,450,29,480
346249,400092743,1,3,1,1,2,46.846,55,715,810,435,490,29,380
346251,400092744,1,3,1,1,2,14.111,20,930,950,570,590,38,310
346255,400093061,1,3,1,1,2,14.437,15,745,800,465,480,31,510
